###**＊＊＊＊＊＊犬と猫を分類するニューラルネットワークモデルを作ってみよう !＊＊＊＊＊＊**

はじめにGPUの設定をしましょう！左上の「編集」->「ノートブックの設定」でランタイムのタイプをPython3、ハードウェアアクセラレーターをGPUに変更して保存してください。

次にDriveをマウントします。下のセルを実行して表示されるURLへアクセスし、コピーしたコードを貼り付けてEnterを押します。これでDrive内のファイルやフォルダにアクセスできるようになります

In [0]:
from google.colab import drive
drive.mount('/content/drive')

マウントが終わったらファイルのディレクトリに移動します。
「My\ Drive」はDriveのトップディレクトリです。ディレクトリは皆さんの環境によって適宜変更してください。

In [0]:
%cd
%cd /content/drive/My\ Drive/EBA/study
%ls
#  nural_network_dog_cat_classification.ipynbが下に表示されることを確認してください

ニューラルネットワークの学習にはKeras(ケラス)というライブラリを利用します。機械学習コードをより簡単に書けるライブラリです（実際学習を実行するのはKerasの内部で動いているTensorflowというライブラリです）

学習に必要なライブラリをインポートします。

In [0]:
import keras
from keras.models import Sequential, Model
from keras.applications import VGG16
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, Input
from keras import optimizers

データの枚数を確認しておきましょう

In [0]:
import os
print(f'犬の学習データ：{len(os.listdir("data/train/dog"))}')
print(f'猫の学習データ：{len(os.listdir("data/train/cat"))}')
print(f'犬の検証データ：{len(os.listdir("data/test/dog"))}')
print(f'猫の検証データ：{len(os.listdir("data/test/cat"))}')

犬と猫の画像を確認しましょう。画像はカラーで、サイズはバラバラです。

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
dog = plt.imread(os.path.join('data/train/dog/', os.listdir('data/train/dog')[0]))
cat = plt.imread(os.path.join('data/train/cat/', os.listdir('data/train/cat')[0]))
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(dog)
ax1.set_xticks([])
ax1.set_yticks([])
ax2 = fig.add_subplot(1,2,2)
ax2.set_xticks([])
ax2.set_yticks([])
ax2.imshow(cat)
plt.show()

画像を事前に学習データと訓練データに分けておきましたね。
下ではそのデータを学習できるように処理をしています。
ここでxは入力（画像データ）、yは正解（犬か猫）です。
yは数字で指定する必要があるので犬＝０、猫＝１としましょう

1.   x_trainという配列に(128, 128, 3)にリサイズ、numpy配列変換した学習データの犬・猫画像を追加します
2.   y_trainという配列に犬の場合は０、猫の場合は１を追加します。

3.   同じように訓練データもx_test, y_test配列に追加します






In [0]:
from PIL import Image
import numpy as np
IMAGE_SHAPE = (128, 128, 3)
x_train = []
y_train = []
x_test = []
y_test = []

# 犬の学習データ
print('processing train dog images... ')
for image in os.listdir('data/train/dog'):
    img_path = os.path.join("data/train/dog", image)
    img = Image.open(img_path)
    img = img.convert('RGB')# 画像をRGBの3次元に変換
    img = img.resize(IMAGE_SHAPE[:2]) # 画像サイズを(128, 128)に変換
    img = np.array(img) #画像をnumpy配列に変換
    x_train.append(img) # 入力として犬の学習画像を追加
    y_train.append(0) #正解として０(犬)を追加

# 猫の学習データ
print('processing train cat images... ')
for image in os.listdir('data/train/cat'):
    img_path = os.path.join("data/train/cat", image)
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize(IMAGE_SHAPE[:2])
    img = np.array(img)
    x_train.append(img) # 入力として猫の学習画像を追加
    y_train.append(1)#正解として1(猫)を追加

# 犬の訓練データ
print('processing test dog images... ')
for image in os.listdir('data/test/dog'):
    img_path = os.path.join('data/test/dog', image)
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize(IMAGE_SHAPE[:2])
    img = np.array(img)
    x_test.append(img)
    y_test.append(0)

print('processing test cat images... ')
# 猫の訓練データ
for image in os.listdir('data/test/cat'):
    img_path = os.path.join('data/test/cat', image)
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize(IMAGE_SHAPE[:2])
    img = np.array(img)
    x_test.append(img)
    y_test.append(1)

さらにデータを処理します。
学習のためにnumpy（ナムパイ　Pythonの計算ライブラリ）の配列の形に変換します。numpyの配列に変換することで行列計算が可能になり画像変換が容易になります

In [0]:
# np.array(配列) で普通の配列からnumpyの配列に変更します
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

# データの形をみてみましょう
print(f"x_train shape is : {x_train.shape}")
print(f"x_test shape is : {x_test.shape}")
print(f"y_train shape is : { y_train.shape}")
print(f"y_test shape is : {y_test.shape}")

# x_trainの1枚目の画像の左上の画素にアクセスしてみましょう
print(f'x_trainの1枚目の画像の左上1pxの画素値: {x_train[0][0,0, :]}')

カラーの画像データは「高さ・幅・画素数」という三種類の形状を持ちます。
(160, 128, 128, 3)というのは高さ128、幅128、画素数3(カラー)の画像が160枚あるという意味です

画素値は0〜255の赤・緑・青から成り立っています。
下のコードでは画素値を0〜255ではなく0〜1の間に収めるために小数点以下が扱えるfloat型に変換して、それぞれの画素を255で割っています。

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # 0~1の値になるように255で割ります（正則化）
x_test /= 255

下のセルを実行すると、先ほどの画素値が0〜1の間に変換されたことがわかります。値を0~1に収めることを正則化と言います。

In [0]:
# x_trainの1枚目の画像の左上の画素にアクセスしてみましょう
print(f'x_trainの1枚目の画像の左上1pxの画素値: {x_train[0][0,0, :]}')

データ処理は終わりました。モデルを作り学習させてみましょう！
画像処理ができるニューラルネットワークをCNN（畳み込みニューラルネットワーク）と言います。スライド資料２章の８以降で次のセルの詳しい説明をしますので確認してください。

In [0]:
#  バッチサイズ（データのグループ）
batch_size = 32
# エポック（学習）
epochs = 50

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=IMAGE_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



ログが出力されましたね。一番下の行の「acc」は学習データの、「val_acc」は検証データのこのモデルの精度です。精度は1 == 100%、全てのデータに対して正解を予測できたということです。

学習が終わったら結果をグラフで表示して見ましょう。青線は学習データの結果、オレンジの線は検証データの結果です。グラフが右肩上がりになり1に近づいていくことが目標です。

In [0]:
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("acc")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

青線の学習データは学習が進むにつれ精度が良くなっているのがわかります。検証データの結果が悪いですね。青線のように右肩上がりになって欲しいところです。またエポックごとの振れ幅が大きいのも気になります。

これで学習は終わりです。スライド資料で学習の解説、モデル改良のためにできることを説明していますので確認してください。

## **＊＊＊＊＊＊番外編＊＊＊＊＊＊**

さて、上で作ったモデルは決していいモデルではありません。検証データに対しての精度が低く過学習です。犬猫の分類は難しくデータ数160枚と少ないので、単純なモデルではうまく機能しないことが多いのです。このままでは学習の凄みがわからないので、手法を変えてモデルを再構築してみます。

In [0]:
# VGG16モデルと学習済み重みをロード
input_tensor = Input(shape=(IMAGE_SHAPE))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# 出力部分の層を構築
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.25))
top_model.add(Dense(1, activation='sigmoid'))

# VGG16と出力部分を接続
model = Model(input=vgg16.input, output=top_model(vgg16.output))

for layer in model.layers[:15]:
    layer.trainable = False


model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("acc")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

グラフを見ると明らかにモデルの精度が上がりましたね。すでに学習済みのモデルの一部を利用する**ファインチューニング**という手法です。今回利用したモデルはVGG16というものです。シンプルなモデルでうまくいかない場合、このような応用技術を用いてみるといいでしょう。興味のある方は調べてみましょう！